In [2]:
import argparse
import os
import sys
import time
import numpy as np

from PIL import Image

import tensorflow as tf
import tensorflow.keras as keras
import matplotlib.pyplot as plt
import urllib
import time

In [3]:
import torch
from functools import partial
from torch import nn, einsum
from einops import repeat, rearrange

In [4]:
def exists(val):
    return val is not Nonev

In [5]:
def map_fn(fn, *args, **kwargs):
    def inner(*arr):
        return map(lambda t: fn(t, *args, **kwargs), arr)
    return inner

In [6]:
def safe_cat_tf(t, el, dim =0):
    if not exists(t):
        return el
    return tf.concat((t, el), axis = dim)

In [7]:
def safe_cat_pt(t, el, dim = 0):
    if not exists(t):
        return el
    return torch.cat((t, el), dim = dim) #concat

# tests

In [8]:
value_dim = 64
key_dim = 64
hidden_dim = 512
output_dim = 4
encoder = None

In [77]:
testA1 = tf.ones((10,2))
testA2 = tf.constant(2.0, shape=(10, 2))
testA1

<tf.Tensor: shape=(10, 2), dtype=float32, numpy=
array([[1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.]], dtype=float32)>

In [78]:
tfC = tf.keras.layers.concatenate([testA1,testA2], axis=-1)
tfC

<tf.Tensor: shape=(10, 4), dtype=float32, numpy=
array([[1., 1., 2., 2.],
       [1., 1., 2., 2.],
       [1., 1., 2., 2.],
       [1., 1., 2., 2.],
       [1., 1., 2., 2.],
       [1., 1., 2., 2.],
       [1., 1., 2., 2.],
       [1., 1., 2., 2.],
       [1., 1., 2., 2.],
       [1., 1., 2., 2.]], dtype=float32)>

In [79]:
tfWk = 2
tfWk * tfC

<tf.Tensor: shape=(10, 4), dtype=float32, numpy=
array([[2., 2., 4., 4.],
       [2., 2., 4., 4.],
       [2., 2., 4., 4.],
       [2., 2., 4., 4.],
       [2., 2., 4., 4.],
       [2., 2., 4., 4.],
       [2., 2., 4., 4.],
       [2., 2., 4., 4.],
       [2., 2., 4., 4.],
       [2., 2., 4., 4.]], dtype=float32)>

In [94]:
tf.math.reduce_sum(tfWk * tfC, axis = 1)

<tf.Tensor: shape=(10,), dtype=float32, numpy=array([12., 12., 12., 12., 12., 12., 12., 12., 12., 12.], dtype=float32)>

In [80]:
c = tf.constant(1.5, shape=(10, 1))
c * tfC

<tf.Tensor: shape=(10, 4), dtype=float32, numpy=
array([[1.5, 1.5, 3. , 3. ],
       [1.5, 1.5, 3. , 3. ],
       [1.5, 1.5, 3. , 3. ],
       [1.5, 1.5, 3. , 3. ],
       [1.5, 1.5, 3. , 3. ],
       [1.5, 1.5, 3. , 3. ],
       [1.5, 1.5, 3. , 3. ],
       [1.5, 1.5, 3. , 3. ],
       [1.5, 1.5, 3. , 3. ],
       [1.5, 1.5, 3. , 3. ]], dtype=float32)>

In [84]:
torchA1 = torch.ones((10,2))
torchA2 = torch.full((10,2),2.0)
torchA1

tensor([[1., 1.],
        [1., 1.],
        [1., 1.],
        [1., 1.],
        [1., 1.],
        [1., 1.],
        [1., 1.],
        [1., 1.],
        [1., 1.],
        [1., 1.]])

In [85]:
torchC = torch.cat((torchA1,torchA2), dim=-1)
torchC

tensor([[1., 1., 2., 2.],
        [1., 1., 2., 2.],
        [1., 1., 2., 2.],
        [1., 1., 2., 2.],
        [1., 1., 2., 2.],
        [1., 1., 2., 2.],
        [1., 1., 2., 2.],
        [1., 1., 2., 2.],
        [1., 1., 2., 2.],
        [1., 1., 2., 2.]])

In [92]:
torchWk = 2
torchWk * torchC

tensor([[2., 2., 4., 4.],
        [2., 2., 4., 4.],
        [2., 2., 4., 4.],
        [2., 2., 4., 4.],
        [2., 2., 4., 4.],
        [2., 2., 4., 4.],
        [2., 2., 4., 4.],
        [2., 2., 4., 4.],
        [2., 2., 4., 4.],
        [2., 2., 4., 4.]])

In [93]:
(torchWk * torchC).sum(dim = 1)

tensor([12., 12., 12., 12., 12., 12., 12., 12., 12., 12.])

In [90]:
twkt = torch.full((10,1),1.5)
twkt * torchC

tensor([[1.5000, 1.5000, 3.0000, 3.0000],
        [1.5000, 1.5000, 3.0000, 3.0000],
        [1.5000, 1.5000, 3.0000, 3.0000],
        [1.5000, 1.5000, 3.0000, 3.0000],
        [1.5000, 1.5000, 3.0000, 3.0000],
        [1.5000, 1.5000, 3.0000, 3.0000],
        [1.5000, 1.5000, 3.0000, 3.0000],
        [1.5000, 1.5000, 3.0000, 3.0000],
        [1.5000, 1.5000, 3.0000, 3.0000],
        [1.5000, 1.5000, 3.0000, 3.0000]])

In [103]:
tlist = []

for i in range(0, 4):
    tlist.append(torch.full((10,1),i, dtype=float))
tlist

[tensor([[0.],
         [0.],
         [0.],
         [0.],
         [0.],
         [0.],
         [0.],
         [0.],
         [0.],
         [0.]], dtype=torch.float64),
 tensor([[1.],
         [1.],
         [1.],
         [1.],
         [1.],
         [1.],
         [1.],
         [1.],
         [1.],
         [1.]], dtype=torch.float64),
 tensor([[2.],
         [2.],
         [2.],
         [2.],
         [2.],
         [2.],
         [2.],
         [2.],
         [2.],
         [2.]], dtype=torch.float64),
 tensor([[3.],
         [3.],
         [3.],
         [3.],
         [3.],
         [3.],
         [3.],
         [3.],
         [3.],
         [3.]], dtype=torch.float64)]

In [104]:
torch.stack(tlist)

tensor([[[0.],
         [0.],
         [0.],
         [0.],
         [0.],
         [0.],
         [0.],
         [0.],
         [0.],
         [0.]],

        [[1.],
         [1.],
         [1.],
         [1.],
         [1.],
         [1.],
         [1.],
         [1.],
         [1.],
         [1.]],

        [[2.],
         [2.],
         [2.],
         [2.],
         [2.],
         [2.],
         [2.],
         [2.],
         [2.],
         [2.]],

        [[3.],
         [3.],
         [3.],
         [3.],
         [3.],
         [3.],
         [3.],
         [3.],
         [3.],
         [3.]]], dtype=torch.float64)

In [105]:
tflist = []

for i in range(0,5):
    tflist.append(tf.constant(i, shape=(10, 1), dtype=float))
    
tflist

[<tf.Tensor: shape=(10, 1), dtype=float32, numpy=
 array([[0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.]], dtype=float32)>,
 <tf.Tensor: shape=(10, 1), dtype=float32, numpy=
 array([[1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.]], dtype=float32)>,
 <tf.Tensor: shape=(10, 1), dtype=float32, numpy=
 array([[2.],
        [2.],
        [2.],
        [2.],
        [2.],
        [2.],
        [2.],
        [2.],
        [2.],
        [2.]], dtype=float32)>,
 <tf.Tensor: shape=(10, 1), dtype=float32, numpy=
 array([[3.],
        [3.],
        [3.],
        [3.],
        [3.],
        [3.],
        [3.],
        [3.],
        [3.],
        [3.]], dtype=float32)>,
 <tf.Tensor: shape=(10, 1), dtype=float32, numpy=
 array([[4.],
        [4.],
        [4.],
        [4.],
        [4.],
        [4.],
        [4.],
        [4.],
      

In [106]:
tf.stack(tflist)

<tf.Tensor: shape=(5, 10, 1), dtype=float32, numpy=
array([[[0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.]],

       [[1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.]],

       [[2.],
        [2.],
        [2.],
        [2.],
        [2.],
        [2.],
        [2.],
        [2.],
        [2.],
        [2.]],

       [[3.],
        [3.],
        [3.],
        [3.],
        [3.],
        [3.],
        [3.],
        [3.],
        [3.],
        [3.]],

       [[4.],
        [4.],
        [4.],
        [4.],
        [4.],
        [4.],
        [4.],
        [4.],
        [4.],
        [4.]]], dtype=float32)>

# img test

In [22]:
allI = []
for i in range(3):
    img_fname = '../imgs/' + str(i) + '.png'
    img = torch.Tensor(np.array(Image.open(img_fname))) / 255.
    allI.append(img)
allI = torch.stack(allI, 0)
allI.shape

torch.Size([3, 32, 32])

In [19]:
tfimg = tf.convert_to_tensor(np.array(Image.open(img_fname)) / 255.)
tfimg.shape

2022-11-20 01:17:55.772888: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-11-20 01:17:55.772937: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2022-11-20 01:17:55.772976: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (jupyter-lak3r): /proc/driver/nvidia/version does not exist
2022-11-20 01:17:55.773483: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


TensorShape([32, 32])

# Einsum test

In [31]:
tf_test_1 = np.random.uniform(size=(1,32,32))
tf_test_2 = np.random.uniform(size=(1,32))
tf_test.shape

(1, 32, 32)

In [33]:
sim = tf.einsum('b n d, b d -> b n',tf_test_1, tf_test_2)
sim                                      

<tf.Tensor: shape=(1, 32), dtype=float64, numpy=
array([[ 7.8576734 ,  9.17129559,  7.89709391,  9.08820142,  9.76681081,
         9.0898406 ,  8.49407793,  8.06102874,  9.44066654,  8.36554971,
         7.92217912,  9.59294428,  9.85872162,  8.63222051,  8.38990892,
         8.69649602,  9.48742956,  8.70055663, 10.28729248,  7.73743186,
         7.03925452,  8.35932872,  8.26403641,  8.66436411,  8.98005011,
         7.24385255,  9.68338367,  8.72796711,  7.73217883,  9.43843247,
         9.23735507,  9.49936075]])>

In [35]:
torch.einsum('b n d, b d -> b n',
             torch.from_numpy(tf_test_1), 
             torch.from_numpy(tf_test_2))

tensor([[ 7.8577,  9.1713,  7.8971,  9.0882,  9.7668,  9.0898,  8.4941,  8.0610,
          9.4407,  8.3655,  7.9222,  9.5929,  9.8587,  8.6322,  8.3899,  8.6965,
          9.4874,  8.7006, 10.2873,  7.7374,  7.0393,  8.3593,  8.2640,  8.6644,
          8.9801,  7.2439,  9.6834,  8.7280,  7.7322,  9.4384,  9.2374,  9.4994]],
       dtype=torch.float64)